In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import platform
import os
from sklearn.utils import class_weight

%matplotlib inline
%load_ext autoreload
%aimport utils
%autoreload 1

In [ ]:
### CACHE FILENAMES ###

train_normal_cache = "training_normal.pickle"
train_pneumonia_cache = "training_pneumonia.pickle"
val_normal_cache = "validation_normal.pickle"
val_pneumonia_cache = "validation_pneumonia.pickle"
test_normal_cache = "testing_normal.pickle"
test_pneumonia_cache = "testing_pneumonia.pickle"

### DATA SETTINGS ###

pdiv = "/" if platform.system() == "Windows" else "\\"

path_train_normal = ".{}chest_xray{}train{}NORMAL{}".format(pdiv, pdiv, pdiv, pdiv)
path_train_pneumonia = ".{}chest_xray{}train{}PNEUMONIA{}".format(pdiv, pdiv, pdiv, pdiv)
path_val_normal = ".{}chest_xray{}val{}NORMAL{}".format(pdiv, pdiv, pdiv, pdiv)
path_val_pneumonia = ".{}chest_xray{}val{}PNEUMONIA{}".format(pdiv, pdiv, pdiv, pdiv)
path_test_normal = ".{}chest_xray{}test{}NORMAL{}".format(pdiv, pdiv, pdiv, pdiv)
path_test_pneumonia = ".{}chest_xray{}test{}PNEUMONIA{}".format(pdiv, pdiv, pdiv, pdiv)

### LABELS ###

normal_label = 0
pneumonia_label = 1
labels = sorted([normal_label, pneumonia_label])

### IMAGE SETTINGS ###

dimension = (64, 64, 3)
resize_dim = dimension[:2]

In [ ]:
### LOAD TRAINING DATA AND LABELS ###

data_train_normal, label_train_normal = utils.load_data(path_train_normal, train_normal_cache, normal_label)
data_train_pneumonia, label_train_pneumonia = utils.load_data(path_train_pneumonia, train_pneumonia_cache, pneumonia_label)

# Caching, disabled for now
# utils.cache(data_train_normal, train_normal_cache)
# utils.cache(data_train_pneumonia, train_pneumonia_cache)

# Combine training images and labels
data_train = np.concatenate((data_train_normal, data_train_pneumonia), axis=0)
label_train = np.asarray(label_train_normal + label_train_pneumonia)

# Shuffle
rand_order = np.random.permutation(data_train.shape[0])
data_train = data_train[rand_order]
label_train = label_train[rand_order]

del data_train_normal
del data_train_pneumonia

In [ ]:
### IMAGE PROCESSING ###

# Image cropping
data_train = utils.images_crop_from_centre(data_train, percentage=0.9)

# Image resizing
data_train = utils.images_resize(data_train, resize_dim) 

# Image normalisation
data_train = utils.images_normalize(data_train)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', np.asarray(labels), label_train)
class_weights = dict(enumerate(class_weights))

In [ ]:
### MODEL PARAMETERS ###
loss_func = "mean_squared_error"
epochs_num = 30
num_of_trainable_layers = 300
model_name = "dense169"
checkpoint_name = "_best_overall"
intermediary_name = model_name + checkpoint_name
ext = ".hdf5"

In [ ]:
### MODEL CREATION ###
from keras.applications.xception import Xception
from keras.applications.mobilenet_v2 import MobileNetV2 
from keras.applications.mobilenet import MobileNet
from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from keras.layers import Dense, GlobalMaxPooling2D, Conv2D, Dropout, GaussianNoise
from keras.models import Model, load_model
from keras.utils import print_summary
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Set model
# base_model = MobileNet(weights='imagenet', include_top=False, pooling='max', input_shape=dimension, alpha=0.75, dropout=0.2)
# base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=dimension, pooling='max')
base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=dimension, pooling='max')
# base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=dimension, pooling='max')

model = base_model.output
model = GaussianNoise(0.4)(model)
model = Dropout(0.5)(model)
model = Dense(32, activation='relu')(model)
model = Dropout(0.5)(model)
model = Dense(16, activation='relu')(model)
preds = Dense(units=1, activation='sigmoid')(model)
model = Model(inputs=base_model.input, outputs=preds)

# Set trainable layers
for layer in model.layers[:-num_of_trainable_layers]:
    layer.trainable = False
for layer in model.layers[-num_of_trainable_layers:]:
    layer.trainable = True

# Callbacks
es_val_loss = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
es_val_acc = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=4)
checkpointer = ModelCheckpoint(intermediary_name + ext, verbose=1, save_best_only=True)
callbacks_list = [es_val_loss, es_val_acc, checkpointer]

# Define optimiser, loss function, and metrics to track
opt = SGD(lr=0.01)
model.compile(optimizer=opt, loss=loss_func, metrics=['accuracy'])

# for i,layer in enumerate(model.layers):
#   print(i, layer.name, layer.output)

In [ ]:
# Model training
history = model.fit(data_train, label_train, epochs=epochs_num, callbacks=callbacks_list, validation_split=0.1, verbose=1, class_weight=class_weights)

In [ ]:
del data_train

In [ ]:
### LOAD VALIDATION DATA AND LABELS ###

data_val_normal, label_val_normal = utils.load_data(path_val_normal, val_normal_cache, normal_label)
data_val_pneumonia, label_val_pneumonia = utils.load_data(path_val_pneumonia, val_pneumonia_cache, pneumonia_label)

# Caching, disabled for now
# utils.cache(data_val_normal, val_normal_cache)
# utils.cache(data_val_pneumonia, val_pneumonia_cache)

# Combine validation images and labels
data_val = np.concatenate((data_val_normal, data_val_pneumonia), axis=0)
label_val = np.asarray(label_val_normal + label_val_pneumonia)

# Shuffle
rand_order = np.random.permutation(data_val.shape[0])
data_val = data_val[rand_order]
label_val = label_val[rand_order]

del data_val_normal
del data_val_pneumonia

In [ ]:
### LOAD TESTING DATA AND LABELS ###

data_test_normal, label_test_normal = utils.load_data(path_test_normal, test_normal_cache, normal_label)
data_test_pneumonia, label_test_pneumonia = utils.load_data(path_test_pneumonia, test_pneumonia_cache, pneumonia_label)

# Caching, disabled for now
# utils.cache(data_test_normal, test_normal_cache)
# utils.cache(data_test_pneumonia, test_pneumonia_cache)

# Combine training images and labels
data_test = np.concatenate((data_test_normal, data_test_pneumonia), axis=0)
label_test = np.asarray(label_test_normal + label_test_pneumonia)

# Shuffle
rand_order = np.random.permutation(data_test.shape[0])
data_test = data_test[rand_order]
label_test = label_test[rand_order]

del data_test_normal
del data_test_pneumonia

In [ ]:
# Image processing for both
data_val = utils.images_crop_from_centre(data_val, percentage=0.9)
data_val = utils.images_resize(data_val, (64, 64)) 
data_val = utils.images_normalize(data_val)

data_test = utils.images_crop_from_centre(data_test, percentage=0.9)
data_test = utils.images_resize(data_test, (64, 64)) 
data_test = utils.images_normalize(data_test)

In [ ]:
# Delete existing model
del model

# Load best version of model
model = load_model(model_name + check.hdf5")

In [ ]:
# Validation and testing
val_loss, val_accuracy = model.evaluate(data_val, label_val)
test_loss, test_accuracy = model.evaluate(data_test, label_test)

print(val_loss, val_accuracy)
print(test_loss, test_accuracy)

In [ ]:
del data_val
del data_test

In [ ]:
# Save the model for usage
model.save(model_name + "_" + str(int(test_accuracy * 100)) + ext)

# Delete intermediary model
if os.path.exists(intermediary_name + ext):
    os.remove(intermediary_name + ext)